# Import

In [ ]:
import os
import numpy as np
import scipy as sp
import tifffile as tiff
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
%matplotlib inline
np.random.seed(1)

from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
from collections import defaultdict
import csv
import sys
csv.field_size_limit(sys.maxsize);

from shapely.wkt import loads as wkt_loads
import random
import datetime
random.seed(0)

from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K

from sklearn.metrics import jaccard_similarity_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Setting

In [ ]:
cwd = os.getcwd()

N_Cls = 10
output = pd.read_csv("../input/sample_submission.csv", dtype={'ImageId':'str','ClassType':'int','MultipolygonWKT':'str'})
image_size = 1024
patch_size = 64
smooth = 1e-12
bands = ['M']

Object Types

1. Building
2. Misc. Manmade Structure
3. Road
4. Track
5. Trees
6. Crops
7. Waterways
8. Standing water
9. Vehicle Large
10. Vehicle Small

# Functions

In [ ]:
class SatelliteImg(object):

    def __init__(self, image_id, high, low, train = True):
        img = self.get_img(image_id)
        self.channel = img.shape[2]
        self.img = np.zeros((image_size,image_size,self.channel))
        # resize image to standard size = 1024*1024
        for i in range(self.channel):   
            image = img[:,:,i].clip(min=low[i], max=high[i])     
            image = 255.0*(image - low[i])/ (high[i]-low[i]) 
            #image = img[:,:,i]
            #
            #self.img[:,:,i] = sp.misc.imresize(image, (image_size,image_size)) 
            self.img[:,:,i] = cv2.resize(image, (image_size,image_size)) 
        self.x_pixel, self.y_pixel = self.img.shape[:2]
        self.img_size = self.x_pixel*self.y_pixel
        for _im_id, _x, _y in csv.reader(open('../input/grid_sizes.csv')):
            if _im_id == image_id:
                self.x_max, self.y_min = float(_x), float(_y)
                break   
        self.x_scaler, self.y_scaler = self.get_scalers() 
        
        if train == True:
            self.train_polygons = [None]*10
            self.train_mask = np.zeros((self.x_pixel, self.y_pixel, N_Cls))
            #self.train_mask = np.zeros((image_size, image_size, N_Cls))
            
            for poly_type in range(0,N_Cls):
                for _im_id, _poly_type, _poly in csv.reader(open('../input/train_wkt_v4.csv')):
                    if _im_id == image_id and _poly_type == str(poly_type+1):                    
                        self.train_polygons[poly_type] = shapely.wkt.loads(_poly)
                        train_polygons_scaled = shapely.affinity.scale(
                            self.train_polygons[poly_type], xfact=self.x_scaler, yfact=self.y_scaler, origin=(0, 0, 0))      
                        self.train_mask[:,:,poly_type] = self.mask_for_polygons(train_polygons_scaled)
                        break

    def stretch_8bit(self, bands, lower_percent=2, higher_percent=98):
        out = np.zeros_like(bands)
        x,y,z = bands.shape
        for i in range(z):
            a = 0 
            b = 255 
            c = np.percentile(bands[:,:,i], lower_percent)
            d = np.percentile(bands[:,:,i], higher_percent)        
            t = a + (bands[:,:,i] - c) * (b - a) / (d - c)    
            t[t<a] = a
            t[t>b] = b
            out[:,:,i] = t
        return out.astype(np.uint8)    
    
    def get_img(self, image_id):
        filename = os.path.join('..', 'input', 'sixteen_band', '{}_M.tif'.format(image_id))
        img = tiff.imread(filename)    
        img = np.rollaxis(img, 0, 3)
        return img
    
    def get_scalers(self):
        w = float(self.y_pixel)
        h = float(self.x_pixel)
        w_ = w * (w / (w + 1))
        h_ = h * (h / (h + 1))
        return w_ / self.x_max, h_ / self.y_min
    
    def mask_for_polygons(self, polygons):
        img_mask = np.zeros((self.x_pixel,self.y_pixel), np.uint8)
        if not polygons:
            return img_mask
        int_coords = lambda x: np.array(x).round().astype(np.int32)
        exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
        interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]
        cv2.fillPoly(img_mask, exteriors, 1)
        cv2.fillPoly(img_mask, interiors, 0)
        return img_mask

    def show_img(self):
        m = self.img
        img = np.zeros_like(m[:,:,0:3])
        img[:,:,0] = m[:,:,4] #red
        img[:,:,1] = m[:,:,2] #green
        img[:,:,2] = m[:,:,1] #blue
        img = self.stretch_8bit(img) # simple strech is probably not a good idea for learning        
        return plt.imshow(img) 
     
    def show_mask(self, poly_type=1):
        #train_mask = self.mask_for_polygons(self.train_polygons_scaled[poly_type])
        train_mask = self.train_mask[:,:,poly_type-1]
        return tiff.imshow(255 * np.stack([train_mask, train_mask, train_mask]));    
    
    def get_polygon(self, poly_type=1):
        return self.train_polygons[poly_type-1]
    
    def get_data(self):
        #img = self.stretch_8bit(self.img)
        img = self.img
        img = img.reshape((self.x_pixel*self.y_pixel, self.channel))
        df = pd.DataFrame(img,index=range(self.img_size))
        return df

    def get_patch(self):
        img = self.img
        mask = self.train_mask   
        x, y, label = [], [], []
        #tr0 = [0.4, 0.1, 0.1, 0.15, 0.3, 0.95, 0.1, 0.05, 0.001, 0.005] # threshold = avg fraction of pixel in each class??
        #tr = [item/2.0 for item in tr0] 
        tr = [0.04, 0.01, 0.009, 0.03, 0.12, 0.27, 0.005, 0.002, 0.00003, 0.00036] 
        no_patch = int(1.0*image_size/patch_size)
        
        for i in range(no_patch):
            for j in range(no_patch):
                xc = i*patch_size
                yc = j*patch_size
                im = img[xc:xc + patch_size, yc:yc + patch_size,:] 
                ms = mask[xc:xc + patch_size, yc:yc + patch_size,:]
                obj = []
                for k in range(N_Cls):
                    sm = np.sum(ms[:, :, k])
                    frac = float(sm) / (patch_size**2)
                    obj.append(frac) 
                #if any(a>b for a,b in zip(obj,tr)):
                if True:
                    x.append(im)
                    y.append(ms)
                    test = [float(m - n)/n for m,n in zip(obj,tr)]
                    label.append(test.index(max(test)))
                  
        return x, y, label

In [ ]:
def mask_to_polygons(mask, epsilon=5., min_area=1.):
    # first, find contours with cv2: it's much faster than shapely
    image, contours, hierarchy = cv2.findContours(((mask == 1) * 255).astype(np.uint8), cv2.RETR_CCOMP, 
                                                  cv2.CHAIN_APPROX_TC89_KCOS)
    # create approximate contours to have reasonable submission size
    approx_contours = [cv2.approxPolyDP(cnt, epsilon, True)
                       for cnt in contours]
    if not contours:
        return MultiPolygon()
    # now messy stuff to associate parent and child contours
    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1
    # http://docs.opencv.org/3.1.0/d9/d8b/tutorial_py_contours_hierarchy.html
    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(approx_contours[idx])
    # create actual polygons filtering by area (removes artifacts)
    all_polygons = []
    for idx, cnt in enumerate(approx_contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = Polygon(
                shell=cnt[:, 0, :],
                holes=[c[:, 0, :] for c in cnt_children.get(idx, [])
                       if cv2.contourArea(c) >= min_area])
            all_polygons.append(poly)
    # approximating polygons might have created invalid ones, fix them
    all_polygons = MultiPolygon(all_polygons)
    if not all_polygons.is_valid:
        all_polygons = all_polygons.buffer(0)
        # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
        # need to keep it a Multi throughout
        if all_polygons.type == 'Polygon':
            all_polygons = MultiPolygon([all_polygons])
    return all_polygons

def to_polygons(mask, instance, epsilon=5., min_area=1.):
    label_img = mask.reshape(instance.x_pixel, instance.y_pixel)
    pred_polygons = mask_to_polygons(label_img, epsilon=epsilon, min_area=min_area)
    scaled_pred_polygons = shapely.affinity.scale(
    pred_polygons, xfact=1 / instance.x_scaler, yfact=1 / instance.y_scaler, origin=(0, 0, 0))
    dumped_prediction = shapely.wkt.dumps(scaled_pred_polygons)
    final_polygons = shapely.wkt.loads(dumped_prediction)
    return final_polygons

In [ ]:
# clipping range for 8 band image (percentile 99.9)
low = [224.0, 190.0, 212.0, 181.0, 100.0, 189.0, 198.0, 138.0]
high = [429.0, 655.0, 937.0, 834.0, 844.0, 836.0, 1162.0, 798.0]

In [ ]:
train_list = ['6010_1_2', '6010_4_2', '6010_4_4', '6040_1_0', '6040_1_3', '6040_2_2', '6040_4_4', '6060_2_3', 
              '6070_2_3', '6090_2_0', '6100_1_3', '6100_2_2', '6100_2_3', '6110_1_2', '6110_3_1', '6110_4_0', 
              '6120_2_0', '6120_2_2', '6140_1_2', '6140_3_1', '6150_2_3', '6160_2_1', '6170_0_4', '6170_2_4', 
              '6170_4_1']

# Getting clipping range

In [ ]:
class SatelliteImg2(object):

    def __init__(self, image_id):
        self.img = self.get_img(image_id)       
        self.x_pixel, self.y_pixel = self.img.shape[:2]
        self.img_size = self.x_pixel*self.y_pixel
        self.channel = self.img.shape[2]
        
    def get_img(self, image_id):
        filename = os.path.join('..', 'input', 'sixteen_band', '{}_M.tif'.format(image_id))
        img = tiff.imread(filename)    
        img = np.rollaxis(img, 0, 3)
        return img
        
    def get_data(self):
        img = self.img
        img = img.reshape((self.x_pixel*self.y_pixel, self.channel))
        df = pd.DataFrame(img,index=range(self.img_size))
        return df

In [ ]:
image_train = []
df_train_part = []

for i, trainable in enumerate(train_list): 
    image_train.append(SatelliteImg2(trainable))
    df_train_part.append(image_train[i].get_data())
    
df_train = pd.concat(df_train_part)

In [ ]:
low = df_train.quantile(q=0.001)
high = df_train.quantile(q=0.999)

for i,column in enumerate(list(df_train.columns.values)):
    df_train[column] = df_train[column].clip(lower=low[i],upper=high[i])

In [ ]:
list(low)

In [ ]:
list(high)

In [ ]:
low = [224.0, 190.0, 212.0, 181.0, 100.0, 189.0, 198.0, 138.0]
high = [429.0, 655.0, 937.0, 834.0, 844.0, 836.0, 1162.0, 798.0]

# Prepare Data

In [ ]:
temp = SatelliteImg('6120_2_2', high = high, low = low )

In [ ]:
image_train = []
train_set = []
target_set = []
label = []

for i, trainable in enumerate(train_list): 
    image_train.append(SatelliteImg(trainable, high = high, low = low))
    x,y, cl = image_train[i].get_patch()   
    train_set += x
    target_set += y
    label += cl
    
train_set = np.array(train_set).transpose(0,3,1,2)
target_set = np.array(target_set).transpose(0,3,1,2)

In [ ]:
train_set.shape

### Make Validation (temporary)

In [ ]:
label_mark = [None]*N_Cls
for search in range(N_Cls):
    label_mark[search] = np.where(np.array(label) == search)[0]

for x in label_mark:
    print len(x)

In [ ]:
train = []
target = []
train_val = []
target_val = []
val_frac = 0.2

for i in range(N_Cls):
    my_list = list(label_mark[i])
    val_size = int(val_frac*len(my_list))
    random.shuffle(my_list)
    for j in range(val_size):
        item_no = my_list.pop()      
        train_val.append(train_set[item_no])
        target_val.append(target_set[item_no])
    for k in my_list:
        train.append(train_set[k])
        target.append(target_set[k])
  
train = np.array(train)
target = np.array(target)
train_val = np.array(train_val)
target_val = np.array(target_val)

In [ ]:
train_val.shape

In [ ]:
train.shape

# Train

In [ ]:
def my_jaccard(img_a,img_b):
    intersection = np.sum(img_a * img_b, axis=(0, -1, -2))
    sum_ = np.sum(img_a + img_b, axis=(0, -1, -2))
    jrk = (intersection + smooth) / (sum_ - intersection + smooth)       
    return jrk

def calc_jacc(model, train, target, finetune = 10):
    img = train
    msk = target

    prd = model.predict(img, batch_size=4)
    jk_list, threshold = [], []

    for i in range(N_Cls):
        t_msk = msk[:, i, :, :]
        t_prd = prd[:, i, :, :]

        m, b_tr = 0, 0
        for j in range(finetune):
            tr = float(j) / finetune
            pred_binary_mask = t_prd > tr
            jk = my_jaccard(t_msk, pred_binary_mask)
            if jk > m:
                m = jk
                b_tr = tr
        #print i, m, b_tr
        jk_list.append(m)
        threshold.append(b_tr)

    score = sum(jk_list) / 10.0
    #print 'jaccard: ',score
    return score, threshold, jk_list

def calc_jacc_thres(model, train, target, threshold= [0.4, 0.1, 0.4, 0.3, 0.3, 0.5, 0.3, 0.6, 0.1, 0.1]):
    img = train
    msk = target

    prd = model.predict(img, batch_size=4)
    jk_list = []

    for i in range(N_Cls):
        t_msk = msk[:, i, :, :]
        t_prd = prd[:, i, :, :]

        tr = threshold[i]
        pred_binary_mask = t_prd > tr
        jk = my_jaccard(t_msk, pred_binary_mask)       
        jk_list.append(jk)

    score = sum(jk_list) / 10.0
    return score, jk_list

In [ ]:
def unet_down(inputs, dim, drop_rate = 0):
    norm = BatchNormalization(axis=1)(inputs)
    drop = Dropout(drop_rate)(norm)
    conv = Convolution2D(dim, 3, 3, activation='relu', border_mode='same')(drop)
    conv = Convolution2D(dim, 3, 3, activation='relu', border_mode='same')(conv)
    pool = MaxPooling2D(pool_size=(2, 2))(conv)
    return conv, pool

def unet_up(inputs, to_merge, dim, drop_rate = 0.2):
    up = merge([UpSampling2D(size=(2, 2))(inputs), to_merge], mode='concat', concat_axis=1)
    norm = BatchNormalization(axis=1)(up)
    drop = Dropout(drop_rate)(norm)
    conv = Convolution2D(dim, 3, 3, activation='relu', border_mode='same')(drop)
    conv = Convolution2D(dim, 3, 3, activation='relu', border_mode='same')(conv)
    return conv

def get_unet():
    inputs = Input((8, patch_size, patch_size))
 
    conv1, layer1 = unet_down(inputs, 32)
    conv2, layer2 = unet_down(layer1, 64)
    conv3, layer3 = unet_down(layer2, 128)
    conv4, layer4 = unet_down(layer3, 256)
    
    norm5 = BatchNormalization(axis=1)(layer4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(norm5)
    layer5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

    layer6 = unet_up(layer5, conv4 , 256,drop_rate=0.1)
    layer7 = unet_up(layer6, conv3, 128,drop_rate=0.2)    
    layer8 = unet_up(layer7, conv2, 64,drop_rate=0.3)
    layer9 = unet_up(layer8, conv1, 32,drop_rate=0.4)
    
    conv10 = Convolution2D(N_Cls, 1, 1, activation='sigmoid')(layer9)

    model = Model(input=inputs, output=conv10)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[jaccard_coef_int])
    return model

def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = y_pred>0.5
    
    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)

def train_model(model, n=1):
    tr_list = np.array([0.0]*10)
    val_tr_list = np.array([0.0]*10)

    for i in range(n):
        print 'epoch:', i+1
        model.fit(train, target, batch_size=16, nb_epoch=1, verbose=2, shuffle=True,
                  validation_data=(train_val,target_val))
        train_score, train_threshold, train_jk_list = calc_jacc(model, train = train, target = target)
        print 'train_jaccard: ', train_score
        val_score, val_threshold, val_jk_list = calc_jacc(model, train = train_val, target = target_val)
        print 'val_jaccard: ', val_score
        tr_list += np.array(train_threshold)
        val_tr_list += np.array(val_threshold)
      
    tr_list /= n
    val_tr_list /=n
    
    return val_score, val_jk_list, tr_list, val_tr_list

In [ ]:
model = get_unet()

In [ ]:
model.optimizer.lr.get_value()

In [ ]:
# good old threshold
val_tr_list = np.array([ 0.386,  0.198,  0.45 ,  0.258,  0.408,  0.514,  0.46 ,  0.192,
        0.008,  0.07 ])
tr_list = np.array([ 0.378,  0.2  ,  0.608,  0.324,  0.414,  0.496,  0.482,  0.442,
        0.098,  0.09 ])
threshold = 0.2*tr_list + 0.8*val_tr_list

In [ ]:
model.fit(train, target, batch_size=16, nb_epoch=100, verbose=1, shuffle=True, validation_data=(train_val,target_val))
model.save_weights('../weights/unet37_100e')
calc_jacc_thres(model, train_val,target_val, threshold=threshold)

In [ ]:
model.fit(train, target, batch_size=16, nb_epoch=25, verbose=1, shuffle=True, validation_data=(train_val,target_val))
model.save_weights('../weights/unet37_125e')
calc_jacc_thres(model, train_val,target_val, threshold=threshold)

In [ ]:
model.load_weights('../weights/unet37_125e')

In [ ]:
model.fit(train, target, batch_size=16, nb_epoch=25, verbose=1, shuffle=True, validation_data=(train_val,target_val))
model.save_weights('../weights/unet37_150e')
calc_jacc_thres(model, train_val,target_val, threshold=threshold)

In [ ]:
tr_list = np.array([ 0.378,  0.2  ,  0.608,  0.324,  0.414,  0.496,  0.482,  0.442,
        0.098,  0.09 ])

In [ ]:
threshold = 0.8*tr_list+0.2*val_tr_list

In [ ]:
score, threshold, jk_list = calc_jacc(model, train = train, target = target)

# Prediction

In [ ]:
from glob import glob
import os
cwd = os.getcwd()

%cd ../input/sixteen_band/
g = glob('*_M.tif')
%cd $cwd
all_list = [item[:8] for item in g]

In [ ]:
output = pd.read_csv("../input/sample_submission.csv", dtype={'ImageId':'str','ClassType':'int','MultipolygonWKT':'str'})

In [ ]:
test_list = [item for item in all_list if item not in train_list]

In [ ]:
def predict(model, img, threshold = [0.4, 0.1, 0.4, 0.3, 0.3, 0.5, 0.3, 0.6, 0.1, 0.1]):
    no_patch = int(1.0*image_size/patch_size)
      
    mask = np.zeros((N_Cls,image_size,image_size))    
        
    for i in range(no_patch):
        for j in range(no_patch):
            xc = i*patch_size
            yc = j*patch_size
            im = img[xc:xc + patch_size, yc:yc + patch_size,:] 
            im = np.transpose(im, (2,0,1))
            im = np.expand_dims(im,axis=0)
            patch_msk = model.predict(im)
            mask[:,xc:xc + patch_size, yc:yc + patch_size] = patch_msk[0,:,:,:]
            
    mask = np.transpose(mask, (1,2,0)) 
    
    for i in range(N_Cls): #predict above threshold
        mask[:,:,i] = mask[:,:,i] > threshold[i]
  
    return mask

In [ ]:
for test in test_list:
    image_test = SatelliteImg(test, high = high, low = low, train = False)
    data = image_test.img
    pred = predict(model, data, threshold)
    print test
    for i in range(N_Cls):        
        result = to_polygons(pred[:,:,i], image_test, epsilon=1., min_area=5.)
        output.set_value((output['ImageId']==test) & (output['ClassType']==i+1),'MultipolygonWKT', str(result))

In [ ]:
output.to_csv('../output/my_unet37_125e.csv', index=False)

In [ ]:
def my_jaccard(img_a,img_b):
    intersection = np.sum(img_a * img_b, axis=(0, -1, -2))
    sum_ = np.sum(img_a + img_b, axis=(0, -1, -2))
    jrk = (intersection + smooth) / (sum_ - intersection + smooth)       
    return jrk
def calc_jacc_thres(model, train, target, threshold= [0.4, 0.1, 0.4, 0.3, 0.3, 0.5, 0.3, 0.6, 0.1, 0.1]):
    img = train
    msk = target

    prd = model.predict(img, batch_size=4)
    jk_list = []

    for i in range(N_Cls):
        t_msk = msk[:, i, :, :]
        t_prd = prd[:, i, :, :]

        tr = threshold[i]
        pred_binary_mask = t_prd > tr
        jk = my_jaccard(t_msk, pred_binary_mask)       
        jk_list.append(jk)

    score = sum(jk_list) / 10.0
    #print 'jaccard: ',score
    return score, jk_list

In [ ]:
calc_jacc_thres(model, train_val, target_val, threshold = threshold)

## second part: building

In [ ]:
output = pd.read_csv("../output/my_unet37_125e.csv", dtype={'ImageId':'str','ClassType':'int','MultipolygonWKT':'str'})
output2 = pd.read_csv("../output/my_unet40_c1_test.csv", dtype={'ImageId':'str','ClassType':'int','MultipolygonWKT':'str'})

In [ ]:
output.head()

In [ ]:
output[output["ClassType"]==1].head()

In [ ]:
output2[output2["ClassType"]==1].head()

In [ ]:
test_list = list(output['ImageId'].unique())

for i,test in enumerate(test_list):
    new = output2[(output2['ImageId']==test) & (output2['ClassType']==1)]["MultipolygonWKT"]
    output.set_value((output['ImageId']==test) & (output['ClassType']==1),'MultipolygonWKT', new)
        
output.to_csv('../output/my_unet37extra.csv', index=False)

In [ ]:
output[output["ClassType"]==1].head()

## Third part: building + waterway + stillwater

In [ ]:
output = pd.read_csv("../output/my_unet37extra.csv", dtype={'ImageId':'str','ClassType':'int','MultipolygonWKT':'str'})
output2 = pd.read_csv("subm/3.csv", dtype={'ImageId':'str','ClassType':'int','MultipolygonWKT':'str'})

In [ ]:
output[(output["ClassType"] == 7) | (output["ClassType"] == 8)].head(10)

In [ ]:
output2[(output2["ClassType"] == 7) | (output2["ClassType"] == 8)].head(10)

In [ ]:
test_list = list(output['ImageId'].unique())

for i,test in enumerate(test_list):
    new = output2[(output2['ImageId']==test) & (output2['ClassType']==7)]["MultipolygonWKT"]
    output.set_value((output['ImageId']==test) & (output['ClassType']==7),'MultipolygonWKT', new)
    new = output2[(output2['ImageId']==test) & (output2['ClassType']==8)]["MultipolygonWKT"]
    output.set_value((output['ImageId']==test) & (output['ClassType']==8),'MultipolygonWKT', new)
        
output.to_csv('../output/my_unet37extra2.csv', index=False)

In [ ]:
output[(output["ClassType"] == 7) | (output["ClassType"] == 8)].head(10)

## Forth part: building, man-made, road, track

In [ ]:
output = pd.read_csv("../output/my_unet37extra.csv", dtype={'ImageId':'str','ClassType':'int','MultipolygonWKT':'str'})
output2 = pd.read_csv("../output/final1.csv", dtype={'ImageId':'str','ClassType':'int','MultipolygonWKT':'str'})

In [ ]:
output[(output["ClassType"] == 2) | (output["ClassType"] == 3) | (output["ClassType"] == 4) | (output["ClassType"] == 5)].head(15)

In [ ]:
output2[(output2["ClassType"] == 2) | (output2["ClassType"] == 3) | (output2["ClassType"] == 4) | (output2["ClassType"] == 5)].head(15)

In [ ]:
test_list = list(output['ImageId'].unique())

for i,test in enumerate(test_list):
    new = output2[(output2['ImageId']==test) & (output2['ClassType']==2)]["MultipolygonWKT"]
    output.set_value((output['ImageId']==test) & (output['ClassType']==2),'MultipolygonWKT', new)
    new = output2[(output2['ImageId']==test) & (output2['ClassType']==3)]["MultipolygonWKT"]
    output.set_value((output['ImageId']==test) & (output['ClassType']==3),'MultipolygonWKT', new)
    new = output2[(output2['ImageId']==test) & (output2['ClassType']==4)]["MultipolygonWKT"]
    output.set_value((output['ImageId']==test) & (output['ClassType']==4),'MultipolygonWKT', new)
        
output.to_csv('../output/final2.csv', index=False)

In [ ]:
output[(output["ClassType"] == 2) | (output["ClassType"] == 3) | (output["ClassType"] == 4) | (output["ClassType"] == 5)].head(15)

# Repair error

In [ ]:
"""
Created on Sat Dec 31 12:22:50 2016

@author: ironbar

Function for dealing with topology exception errors
"""
import pandas as pd
import numpy as np
import time
import shapely
import shapely.geometry

def __get_valid_wkt_str(input, precision, debug_print):
    """
    Function that checks that a wkt str is valid
    """
    if type(input) is str:
        wkt_str = input
    else:
        #Get the initial str
        wkt_str = shapely.wkt.dumps(input, rounding_precision=precision)
    #Loop until we find a valid polygon
    for i in range(100):
        polygon = shapely.wkt.loads(wkt_str)
        if not polygon.is_valid:
            #print debugging info
            print 'Invalid polygon in %s. Iteration: %i' % (debug_print, i)
            #Use buffer to try to fix the polygon
            polygon = polygon.buffer(0)
            #Get back to multipolygon if necesary
            if polygon.type == 'Polygon':
                polygon = shapely.geometry.MultiPolygon([polygon])
            #Dump to str again
            wkt_str = shapely.wkt.dumps(polygon, rounding_precision=precision)
        else:
            break
    return wkt_str

    
def __create_square_around_point(point, side):
    """
    Creates a square polygon with shapely given 
    the center point
    """
    #Create canonical square points
    square_points = np.zeros((4,2))
    square_points[1, 0] = 1
    square_points[2] = 1
    square_points[3, 1] = 1
    #Scale the square
    square_points *= side
    #Position to have the point in the center
    for i in range(2):
        square_points[:, i] += point[i] - side/2.
    pol = shapely.geometry.Polygon(square_points)
    return pol    
    
def repair_topology_exception(submission_path, precision, image_id, n_class, point, 
                              side=1e-4):
    """
    Tries to repair the topology exception error by creating a squared 
    hole in the given point with the given side
    """
    start_time = time.time()
    #Load the submission
    print 'Loading the submission...'
    df = pd.read_csv(submission_path)
    #Loop over the values changing them
    #I'm going to use the values because I think iterating over the dataframe is slow
    #But I'm not sure
    print 'Looping over the polygons'
    polygons = df.MultipolygonWKT.values
    img_ids = df.ImageId.values
    class_types = df.ClassType.values
    for i, polygon, img_id, class_type in zip(range(len(polygons)), polygons,
                                           img_ids, class_types):
        if img_id == image_id and n_class == class_type:
            polygon = shapely.wkt.loads(polygon)
            square = __create_square_around_point(point, side)
            polygon = polygon.difference(square)
            polygons[i] = __get_valid_wkt_str(polygon, precision,
                '%s class%i' % (img_id, class_type))
    #Update the dataframe
    df.MultipolygonWKT = polygons
    #Save to a new file
    print 'Saving the submission...'
    df.to_csv(submission_path,
              index=False)
    print 'It took %i seconds to repair the submission' % (
        time.time() - start_time)


In [ ]:
repair_topology_exception('../output/my_unet37_125e.csv', 
                              precision=6, 
                              image_id='6100_0_2', 
                              n_class=2, 
                              point= (0.0080394952501907386 , -0.0034979715011444375), 
                              side=1e-4)

# Visualization

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

output = pd.read_csv("../input/sample_submission.csv", dtype={'ImageId':'str','ClassType':'int','MultipolygonWKT':'str'})
test_list = list(output['ImageId'].unique())

model.load_weights('../weights/unet37_125e')

# good old threshold
val_tr_list = np.array([ 0.386,  0.198,  0.45 ,  0.258,  0.408,  0.514,  0.46 ,  0.192,
        0.008,  0.07 ])
tr_list = np.array([ 0.378,  0.2  ,  0.608,  0.324,  0.414,  0.496,  0.482,  0.442,
        0.098,  0.09 ])
threshold = 0.2*tr_list + 0.8*val_tr_list

In [ ]:
class SatelliteImg(object):

    def __init__(self, image_id, high, low, train = True):
        self.img = np.empty((image_size,image_size,0))
        self.img2 = np.empty((image_size,image_size,0))
        j = 0
        for band in bands:
            img = self.get_img(image_id, band)
            channel = img.shape[2]
            img_resized1 = np.zeros((image_size,image_size,channel))
            img_resized2 = np.zeros((image_size,image_size,channel))
            # resize image to standard size = 1024*1024
            for i in range(channel):
                img_resized1[:,:,i] = cv2.resize(img[:,:,i], (image_size,image_size))
                img_resized2[:,:,i] = img_resized1[:,:,i]
                img_resized1[:,:,i] = img_resized1[:,:,i].clip(min=low[j], max=high[j])     
                img_resized1[:,:,i] = 255.0*(img_resized1[:,:,i] - low[j])/ (high[j]-low[j]) 
                j += 1          
            self.img = np.append(self.img, img_resized1, axis=-1)
            self.img2 = np.append(self.img2, img_resized2, axis = -1)
        
        self.channel = self.img.shape[2]
        self.x_pixel, self.y_pixel = self.img.shape[:2]
        self.img_size = self.x_pixel*self.y_pixel
        for _im_id, _x, _y in csv.reader(open('../input/grid_sizes.csv')):
            if _im_id == image_id:
                self.x_max, self.y_min = float(_x), float(_y)
                break   
        self.x_scaler, self.y_scaler = self.get_scalers() 
        
        self.train_polygons = [None]*10
        self.train_polygons_scaled = [None]*10
        self.train_mask = np.zeros((self.x_pixel, self.y_pixel, N_Cls))
        
        if train == True:
            for poly_type in range(0,N_Cls):
                for _im_id, _poly_type, _poly in csv.reader(open('../input/train_wkt_v4.csv')):
                    if _im_id == image_id and _poly_type == str(poly_type+1):                    
                        self.train_polygons[poly_type] = shapely.wkt.loads(_poly)
                        self.train_polygons_scaled[poly_type] = shapely.affinity.scale(
                            self.train_polygons[poly_type], xfact=self.x_scaler, yfact=self.y_scaler, origin=(0, 0, 0))      
                        self.train_mask[:,:,poly_type] = self.mask_for_polygons(self.train_polygons_scaled[poly_type])
                        break

    def stretch_8bit(self, bands, lower_percent=2, higher_percent=98):
        out = np.zeros_like(bands)
        x,y,z = bands.shape
        for i in range(z):
            a = 0 
            b = 255 
            c = np.percentile(bands[:,:,i], lower_percent)
            d = np.percentile(bands[:,:,i], higher_percent)        
            t = a + (bands[:,:,i] - c) * (b - a) / (d - c)    
            t[t<a] = a
            t[t>b] = b
            out[:,:,i] = t
        return out.astype(np.uint8)    
    
    def get_img(self, image_id, band = 'M'):
        if band == 'M':
            filename = os.path.join('..', 'input', 'sixteen_band', '{}_M.tif'.format(image_id))
        elif band == 'A':
            filename = os.path.join('..', 'input', 'sixteen_band', '{}_A.tif'.format(image_id)) 
        elif band == 'P':
            filename = os.path.join('..', 'input', 'sixteen_band', '{}_P.tif'.format(image_id))
        elif band == 'rgb':
            filename = os.path.join('..', 'input', 'three_band', '{}.tif'.format(image_id))
        img = tiff.imread(filename) 
        if band == 'P':
            img = np.expand_dims(img, axis=0)
        img = np.rollaxis(img, 0, 3)
        return img
    
    def get_scalers(self):
        w = float(self.y_pixel)
        h = float(self.x_pixel)
        w_ = w * (w / (w + 1))
        h_ = h * (h / (h + 1))
        return w_ / self.x_max, h_ / self.y_min
    
    def mask_for_polygons(self, polygons):
        img_mask = np.zeros((self.x_pixel,self.y_pixel), np.uint8)
        if not polygons:
            return img_mask
        int_coords = lambda x: np.array(x).round().astype(np.int32)
        exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
        interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]
        cv2.fillPoly(img_mask, exteriors, 1)
        cv2.fillPoly(img_mask, interiors, 0)
        return img_mask

    def show_img(self):
        m = self.img
        img = np.zeros_like(m[:,:,0:3])
        img[:,:,0] = m[:,:,4] #red
        img[:,:,1] = m[:,:,2] #green
        img[:,:,2] = m[:,:,1] #blue
        img = self.stretch_8bit(img) # simple strech is probably not a good idea for learning        
        return plt.imshow(img) 
    
    def get_mask(self):
        for i in range(N_Cls):
            self.train_polygons_scaled[i] = shapely.affinity.scale(
                            self.train_polygons[i], xfact=self.x_scaler, yfact=self.y_scaler, origin=(0, 0, 0))
            self.train_mask[:,:,i] = self.mask_for_polygons(self.train_polygons_scaled[i])
        return self.train_mask
    
    def show_mask(self, poly_type=1):
        #train_mask = self.mask_for_polygons(self.train_polygons_scaled[poly_type])
        train_mask = self.train_mask[:,:,poly_type]
        return tiff.imshow(255 * np.stack([train_mask, train_mask, train_mask]))   
    
    def get_polygon(self, poly_type=1):
        return self.train_polygons[poly_type]
    
    def get_data(self):
        #img = self.stretch_8bit(self.img)
        img = self.img
        img = img.reshape((self.x_pixel*self.y_pixel, self.channel))
        df = pd.DataFrame(img,index=range(self.img_size))
        return df

    def get_patch(self):
        img1 = self.img
        #img2 = self.img2
        mask = self.train_mask   
        #x1, x2, y, label = [],[], [], []
        x1, y, label = [],[], []
        tr = [0.04, 0.01, 0.009, 0.03, 0.12, 0.27, 0.005, 0.002, 0.00003, 0.00036] 
        no_patch = int(1.0*image_size/patch_size)
        
        for i in range(no_patch):
            for j in range(no_patch):
                xc = i*patch_size
                yc = j*patch_size
                im1 = img1[xc:xc + patch_size, yc:yc + patch_size,:] 
                #im2 = img2[xc:xc + patch_size, yc:yc + patch_size,:] 
                ms = mask[xc:xc + patch_size, yc:yc + patch_size,:]
                obj = []
                for k in range(N_Cls):
                    sm = np.sum(ms[:, :, k])
                    frac = float(sm) / (patch_size**2)
                    obj.append(frac) 
                x1.append(im1)
                #x2.append(im2)
                y.append(ms)
                test = [float(m - n)/n for m,n in zip(obj,tr)]
                label.append(test.index(max(test)))
                  
        #return x1,x2, y, label
        return x1, y, label

cdict = {'red':[255,1,1],'purple':[200,40,194],'grey':[128,128,128],'brown':[102,51,1],'green':[75,212,65],
         'ggreen':[204,155,153],'blue':[56,88,215],'navy':[1,1,102],'yellow':[153,153,1],'yyellow':[255,255,102]}

def color(img, rgb=cdict['grey']):
    result = np.zeros((1024,1024,4))
    result[:,:,0:3] +=  np.transpose(np.stack([rgb[0]*img, rgb[1]*img, rgb[2]*img]),(1,2,0))/255.0
    result[:,:,3] += 0.7*img
       
    return result.astype(np.float32)

def stretch_8bit(bands, lower_percent=2, higher_percent=98):
    out = np.zeros_like(bands)
    x,y,z = bands.shape

    for i in range(z):
        a = 0 
        b = 255 
        c = np.percentile(bands[:,:,i], lower_percent)
        d = np.percentile(bands[:,:,i], higher_percent)        
        t = a + (bands[:,:,i] - c) * (b - a) / (d - c)    
        t[t<a] = a
        t[t>b] = b
        out[:,:,i] = t
        
    if out.shape[2] != 1:
        output = out.astype(np.uint8) 
    else:
        output = out[:,:,0].astype(np.uint8) 
    return output

def get_rgb(image_id):
    img = tiff.imread("../input/three_band/{}.tif".format(image_id))
    img = np.transpose(img,(1,2,0))
    img = stretch_8bit(img)
    rgb = np.zeros((image_size,image_size,3))
    
    for i in range(3):
        rgb[:,:,i] = cv2.resize(img[:,:,i], (image_size,image_size)) 
        
    return rgb.astype(np.uint8)

def get_color(instance):
    dd = []
    msk = instance.get_mask()
    dd.append(color(msk[:,:,0],cdict['red']))
    dd.append(color(msk[:,:,1],cdict['purple']))
    dd.append(color(msk[:,:,2],cdict['grey']))
    dd.append(color(msk[:,:,3],cdict['brown']))
    dd.append(color(msk[:,:,4],cdict['green']))
    dd.append(color(msk[:,:,5],cdict['ggreen']))
    dd.append(color(msk[:,:,6],cdict['blue']))
    dd.append(color(msk[:,:,7],cdict['navy']))
    dd.append(color(msk[:,:,8],cdict['yellow']))
    dd.append(color(msk[:,:,9],cdict['yyellow']))
    
    return dd

def show_pics(image_id, dd, msk_no=0, y1=0,y2=image_size,x1=0,x2=image_size):
    f, (real, mask) = plt.subplots(1, 2, figsize=(15,8))
  
    my_rgb = get_rgb(image_id)
    real.imshow(my_rgb[y1:y2,x1:x2])
    mask.imshow(my_rgb[y1:y2,x1:x2])
    mask.imshow(dd[msk_no-1][y1:y2,x1:x2])
    
def show_pics_all(image_id, y1=0,y2=image_size,x1=0,x2=image_size):
    f, (real, mask) = plt.subplots(1, 2, figsize=(30,16))

    image_test = SatelliteImg(image_id, high = high, low = low, train = False)
    data = image_test.img
    pred = predict(model, data, threshold)

    for i in range(N_Cls):        
        image_test.train_polygons[i] = to_polygons(pred[:,:,i], image_test, epsilon=1., min_area=5.)
    dd = get_color(image_test)

    my_rgb = get_rgb(image_id)
    real.imshow(my_rgb[y1:y2,x1:x2])
    mask.imshow(my_rgb[y1:y2,x1:x2])
    mask.imshow(dd[0][y1:y2,x1:x2])
    mask.imshow(dd[1][y1:y2,x1:x2])
    mask.imshow(dd[2][y1:y2,x1:x2])
    mask.imshow(dd[3][y1:y2,x1:x2])
    mask.imshow(dd[4][y1:y2,x1:x2])
    mask.imshow(dd[5][y1:y2,x1:x2])
    mask.imshow(dd[6][y1:y2,x1:x2])
    mask.imshow(dd[7][y1:y2,x1:x2])
    mask.imshow(dd[8][y1:y2,x1:x2])
    mask.imshow(dd[9][y1:y2,x1:x2])
    
    f.suptitle(image_id)
    
    f.savefig('../visualization_pred37/{}.jpg'.format(image_id))
    
def show_pics_label(image_id, y1=0,y2=image_size,x1=0,x2=image_size):
    f, (real, mask) = plt.subplots(1, 2, figsize=(30,16))

    image_test = SatelliteImg(image_id, high = high, low = low, train = True)
    dd = get_color(image_test)

    my_rgb = get_rgb(image_id)
    real.imshow(my_rgb[y1:y2,x1:x2])
    mask.imshow(my_rgb[y1:y2,x1:x2])
    mask.imshow(dd[0][y1:y2,x1:x2])
    mask.imshow(dd[1][y1:y2,x1:x2])
    mask.imshow(dd[2][y1:y2,x1:x2])
    mask.imshow(dd[3][y1:y2,x1:x2])
    mask.imshow(dd[4][y1:y2,x1:x2])
    mask.imshow(dd[5][y1:y2,x1:x2])
    mask.imshow(dd[6][y1:y2,x1:x2])
    mask.imshow(dd[7][y1:y2,x1:x2])
    mask.imshow(dd[8][y1:y2,x1:x2])
    mask.imshow(dd[9][y1:y2,x1:x2])
    f.suptitle(image_id)
    
    f.savefig('../visualization_label/{}.jpg'.format(image_id))
    
def show_all_label(img_list):

    for i,img in enumerate(img_list):
        show_pics_label(img) 

def show_all_pred(img_list):

    for i,img in enumerate(img_list):
        show_pics_all(img)         

In [ ]:
show_pics_all('6100_0_2')

In [ ]:
show_all_label(train_list)

In [ ]:
from PIL import Image

def merge_images(file_list):
    file_list.sort()
    no_img = len(file_list)
    test = Image.open('../visualization_label/6010_1_2.jpg')
    (width, height) = test.size
    result = Image.new('RGB', (width, no_img*height))
    
    
    for i,myfile in enumerate(file_list):
        print i
        img = Image.open('../visualization_pred37/{}.jpg'.format(myfile))
        result.paste(im=img, box=(0, i*height))
 
    result.save('../visualization_pred37/all.jpg')


In [ ]:
merge_images(test_list)

In [ ]:
show_all_pred(test_list[358:])

In [ ]:
test_list.index('6160_0_0')

In [ ]:
test_list.sort()

In [ ]:
%cd ../visualization_pred37/
g = glob('*.*')
%cd $cwd

In [ ]:
len(g)